<a href="https://colab.research.google.com/github/WilliamsOrigin/DataMiningProject/blob/main/project_bigdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Big Data Analytics Project proposal 2021 - 2022

In [479]:
# install java 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [480]:
# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [481]:
# unzip the spark file to the current folder 
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

Set the environment path which will enables to run Pyspark in Colab environment.

In [482]:
# set your spark folder to your system path environment. 
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [483]:
# install pyspark and findspark using pip 
!pip install pyspark
!pip install -q findspark

In [549]:
## for data
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import statistics
import operator


Run a local spark session to test the installation and then import all the input files containing the data of the project.

In [550]:
from numpy import array
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.mllib.linalg import Vectors, SparseMatrix
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Row
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import Imputer


In [557]:
from pyspark.ml.feature import PCA
from sklearn.preprocessing import StandardScaler


In [486]:
#conf = pyspark.SparkConf().setAppName('BigData').setMaster('local')
#sc = pyspark.SparkContext(conf=conf)

In [487]:

spark = SparkSession(sc)

## Import Files

In [531]:
sc = spark.sparkContext
lines = sc.textFile('sample_data/pendigits.txt')

def split_line(line):
  
  #parts_t = list(map(split_part, line))
  parts = line.split(" ")
  res=[np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN ]
  for part in parts:
    p = part.split(":")
    if(len(p) == 1):
      res[0] = int(p[0])
    else:
      res[int(p[0])] = int(p[1])
  
  return res

parts = lines.map(split_line)


In [532]:
data = parts.map(lambda p: Row(target=p[0], a=p[1], b=p[2], c=p[3],d=p[4], e=p[5], f=p[6], g=p[7], h=p[8], i=p[9], j=p[10], k=p[11], l=p[12], m=p[13], n=p[14], o=p[15], p=p[16]))
dbf = spark.createDataFrame(data)
dbf.show()

+------+----+---+---+---+----+---+---+----+----+---+----+----+---+----+----+----+
|target|   a|  b|  c|  d|   e|  f|  g|   h|   i|  j|   k|   l|  m|   n|   o|   p|
+------+----+---+---+---+----+---+---+----+----+---+----+----+---+----+----+----+
|     8|  47|100| 27| 81|  57| 37| 26| NaN| NaN| 23|  56|  53|100|  90|  40|  98|
|     2|null| 89| 27|100|  42| 75| 29|null|null| 15|  37|null| 69|   2| 100|   6|
|     1|null| 57| 31| 68|  72| 90|100|null|null| 75|  50|  51| 28|  25|  16|null|
|     4|null|100|  7| 92|   5| 68| 19|null|null| 34| 100|  45| 74|  23|  67|null|
|     1|null| 67| 49| 83| 100|100| 81|null|null| 60|  40|  40| 33|  20|  47|null|
|     6| 100|100| 88| 99|  49| 74| 17|null| NaN| 16|  37|null| 73|  16|  20|  20|
|     4|null|100|  3| 72|  26| 35| 85|null|null| 71|  73|  97| 65|  49|  66|null|
|     0|null| 39|  2| 62|  11|  5| 63| NaN|null| 43|  89|  99| 36| 100|null|  57|
|     5|  13| 89| 12| 50|  72| 38| 56| NaN|null| 17|null|  61| 32|  94| 100| 100|
|     0|  57|100

## 1- Clean the Dataset

In [533]:
imputer = Imputer()
imputer.setInputCols(["a","b", "c","d","e","f","g","j","k","l","m","n","o","p"])
imputer.setOutputCols(["out_a", "out_b",  "out_c","out_d","out_e","out_f","out_g","out_j","out_k","out_l","out_m","out_n","out_o","out_p"])
imputer.getRelativeError()
model = imputer.fit(dbf)

In [515]:
model.surrogateDF.show()

+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+
|                a|                b|                c|               d|                e|                f|                 g|                j|                k|                l|                 m|                 n|                o|                p|
+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+
|48.11231324059763|84.97415640064274|43.88844971453667|83.0224538893344|59.51951295660319|67.63466074649646|56.343850423605026|42.55233333333333|69.33745103946973|40.13035019455253|55.801796896270076|42.204027056787794|63.3595810005

In [534]:
mdbf = model.transform(dbf)

In [535]:
mdbf = mdbf.drop(*["a","b", "c","d","e","f","g","h","i","j","k","l","m","n","o","p"])
mdbf.show()

+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|target|out_p|out_e|out_l|out_d|out_m|out_c|out_n|out_j|out_g|out_b|out_o|out_k|out_a|out_f|
+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|     8|   98|   57|   53|   81|  100|   27|   90|   23|   26|  100|   40|   56|   47|   37|
|     2|    6|   42|   40|  100|   69|   27|    2|   15|   29|   89|  100|   37|   48|   75|
|     1|   45|   72|   51|   68|   28|   31|   25|   75|  100|   57|   16|   50|   48|   90|
|     4|   45|    5|   45|   92|   74|    7|   23|   34|   19|  100|   67|  100|   48|   68|
|     1|   45|  100|   40|   83|   33|   49|   20|   60|   81|   67|   47|   40|   48|  100|
|     6|   20|   49|   40|   99|   73|   88|   16|   16|   17|  100|   20|   37|  100|   74|
|     4|   45|   26|   97|   72|   65|    3|   49|   71|   85|  100|   66|   73|   48|   35|
|     0|   57|   11|   99|   62|   36|    2|  100|   43|   63|   39|  

In [551]:
corr_list ={}
for col in ["out_a", "out_b",  "out_c","out_d","out_e","out_f","out_g","out_j","out_k","out_l","out_m","out_n","out_o","out_p"]:
  corr_list["target_" + col] = mdbf.stat.corr("target", col)


Out_a (col 1) variable is more correlated with target

In [553]:
corr_list_sort = sorted(corr_list.items(),key = operator.itemgetter(1),reverse = True)
corr_list_sort

[('target_out_a', 0.38674488228697257),
 ('target_out_b', 0.25263699832435726),
 ('target_out_c', 0.22381247898401008),
 ('target_out_d', 0.18250138663367413),
 ('target_out_j', 0.1341639378050535),
 ('target_out_m', 0.09033156450219507),
 ('target_out_f', 0.0834792785060136),
 ('target_out_p', 0.023040108995573384),
 ('target_out_e', 0.005053615780746761),
 ('target_out_o', 0.004769029821385412),
 ('target_out_n', -0.0565327399594594),
 ('target_out_k', -0.06538807705345413),
 ('target_out_l', -0.10772965905197994),
 ('target_out_g', -0.12498477823514045)]

**Visualize data in two dimensions by going through PCA**

In [562]:
#PCA
feature = mdbf.drop(*["target"])

